# Analyse des abstract

Question : quels sont les intellectuels qui ont un lien avec le féminisme ?

Ici nous récupérons les résumés disponibles sur wikipédia pour chaque personne (et uniquement sur les pages en anglais) et nous cherchons celles pour lesquelles les termes "feminist" ou "feminism" apparaissent.

### Importation des librairies

In [1]:
from SPARQLWrapper import SPARQLWrapper, SPARQLWrapper2, JSON, TURTLE, XML, RDFXML
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np

In [2]:
import pprint
import csv
import ast
import re

from collections import Counter
from operator import itemgetter

import sqlite3 as sql
import time

from importlib import reload
from shutil import copyfile

In [3]:
from ipywidgets import (
    interact, IntSlider, Checkbox,
    interactive, SelectMultiple)

In [4]:
import sparql_functions as spqf

In [5]:
reload(spqf)

<module 'sparql_functions' from 'C:\\jupyter_notebooks\\JS_intellectuels\\sparql_functions.py'>

## Intellectuels en lien avec le féminisme

In [6]:
db = 'data/intellectuels.db'

In [7]:
abstract = """
SELECT GROUP_CONCAT(value), uri_entity, count(*) as eff
FROM property p
WHERE p.property LIKE '%abstract%' 
GROUP BY uri_entity
ORDER BY eff DESC ;

"""

In [8]:
# Connnexion à la base de données
cn = sql.connect(db)
c = cn.cursor()

### Exécuter la requête sur la base de donées SQLite pour récupérer les valeurs que contient la ligne
c.execute(abstract)
result_q = []
result_q = c.fetchall()

# Fermer la connexion et afficher les cinq premiers résultats
cn.close()
print(len(result_q))
result_q[:5]

185583


[("Abd al-'Aziz ibn Muhammad al-Qahiri al-Wafa'i known as Ibn al-Aqba'i (1408–1471) was an Egyptian astronomer and mathematician in the 15th century. He was born in 811 H.E. (1408 AD) and died in 876 H.E (1471 AD). Some sources say that he died in 874 or 879 H.E. He was a student of Ibn al-Majdi and . He became a time keeper at the Mu'ayyad Mosque in Cairo. He also became the chief of the time keepers of the Azhar mosque and the Maridani Mosque. He was interested in the calculation of the local times of cities based on their latitude and longitude. He invented an astronomical device called Da'irah al-Mu'addal. He wrote about 40 books and 20 treatises on mathematics and astronomy, none of them published. Among his works is the manuscript Risalah al-'Amal bi al-Rub' al-Mujayyab",
  "http://dbpedia.org/resource/'Abd_al-'Aziz_al-Wafa'i",
  1),
 ("'Abd al-Razzaq al-Hasani (Arabic: عبد الرزاق الحسني\u200e) (1903–1997) was an Iraqi historian and politician. Al-Hasani was a prominent proponent

In [9]:
df_abs = pd.DataFrame(result_q, columns=['abstract','person','request'])
df_abs

,abstract,person,request
0,Abd al-'Aziz ibn Muhammad al-Qahiri al-Wafa'i ...,http://dbpedia.org/resource/'Abd_al-'Aziz_al-W...,1
1,'Abd al-Razzaq al-Hasani (Arabic: عبد الرزاق ا...,http://dbpedia.org/resource/'Abd_al-Razzaq_al-...,1
2,'Ali ibn al-'Abbas al-Majusi (Persian: علی بن ...,http://dbpedia.org/resource/'Ali_ibn_al-'Abbas...,1
3,The 14th Dalai Lama (religious name: Tenzin Gy...,http://dbpedia.org/resource/14th_Dalai_Lama,1
4,Matthew and Grace Huang are an American couple...,http://dbpedia.org/resource/2013_Huang_Case_in...,1
...,...,...,...
185578,"Ștefan Secăreanu (born January 8, 1959, Chiose...",http://dbpedia.org/resource/Ștefan_Secăreanu,1
185579,"Ștefan Voitec (also rendered Ștefan Voitech, S...",http://dbpedia.org/resource/Ștefan_Voitec,1
185580,"Ștefan Zeletin (born Ștefan Motăș; June 19, 18...",http://dbpedia.org/resource/Ștefan_Zeletin,1
185581,Dr. ʻAna Maui Taufeʻulungaki is a Tongan acade...,http://dbpedia.org/resource/ʻAna_Taufeʻulungaki,1


REMARQUE : sur les 197'176 personnes, 185'583 ont un résumé en anglais sur wikipédia.

In [10]:
searchfor = ['feminist', 'feminism']
contain_feminism = df_abs[df_abs['abstract'].str.contains('|'.join(searchfor))]
contain_feminism

,abstract,person,request
212,Agampodi Paulus de Zoysa (5 April 1890 – 26 Ma...,http://dbpedia.org/resource/A._P._de_Zoysa,1
480,"Aasta Hansteen, also known as Hasta Hanseen (b...",http://dbpedia.org/resource/Aasta_Hansteen,1
835,Abdullah Öcalan ( OH-jə-lahn; Turkish: [œdʒaɫa...,http://dbpedia.org/resource/Abdullah_Öcalan,1
1540,"Adam Lewis ""A.L."" Bingaman (born c. 1790; died...",http://dbpedia.org/resource/Adam_Lewis_Bingaman,1
1701,"Addie Wagenknecht (born Portland, Oregon) is a...",http://dbpedia.org/resource/Addie_Wagenknecht,1
...,...,...,...
184439,Zaib-un-Nissa (Bengali: জেবুন্নেসা হামিদুল্লাহ...,http://dbpedia.org/resource/Zaib-un-Nissa_Hami...,1
184642,"Zenna Chlarson Henderson (November 1, 1917 – M...",http://dbpedia.org/resource/Zenna_Henderson,1
184905,Zoe Fairbairns (born 1948) is a British femini...,http://dbpedia.org/resource/Zoe_Fairbairns,1
185086,"Ásta Kristjana Sveinsdóttir (born October 5, 1...",http://dbpedia.org/resource/Ásta_Kristjana_Sve...,1


REMARQUE : sur ces 185'583, 899 contiennent les mots "feminism" ou "feminist" dans leur résumé wiki

In [11]:
# Crée un tableau montrant abstract et person
df = contain_feminism[['abstract','person']]
df

,abstract,person
212,Agampodi Paulus de Zoysa (5 April 1890 – 26 Ma...,http://dbpedia.org/resource/A._P._de_Zoysa
480,"Aasta Hansteen, also known as Hasta Hanseen (b...",http://dbpedia.org/resource/Aasta_Hansteen
835,Abdullah Öcalan ( OH-jə-lahn; Turkish: [œdʒaɫa...,http://dbpedia.org/resource/Abdullah_Öcalan
1540,"Adam Lewis ""A.L."" Bingaman (born c. 1790; died...",http://dbpedia.org/resource/Adam_Lewis_Bingaman
1701,"Addie Wagenknecht (born Portland, Oregon) is a...",http://dbpedia.org/resource/Addie_Wagenknecht
...,...,...
184439,Zaib-un-Nissa (Bengali: জেবুন্নেসা হামিদুল্লাহ...,http://dbpedia.org/resource/Zaib-un-Nissa_Hami...
184642,"Zenna Chlarson Henderson (November 1, 1917 – M...",http://dbpedia.org/resource/Zenna_Henderson
184905,Zoe Fairbairns (born 1948) is a British femini...,http://dbpedia.org/resource/Zoe_Fairbairns
185086,"Ásta Kristjana Sveinsdóttir (born October 5, 1...",http://dbpedia.org/resource/Ásta_Kristjana_Sve...


In [17]:
# Crée un tableau montrant person
df = contain_feminism[['person']]
df

,person
212,http://dbpedia.org/resource/A._P._de_Zoysa
480,http://dbpedia.org/resource/Aasta_Hansteen
835,http://dbpedia.org/resource/Abdullah_Öcalan
1540,http://dbpedia.org/resource/Adam_Lewis_Bingaman
1701,http://dbpedia.org/resource/Addie_Wagenknecht
...,...
184439,http://dbpedia.org/resource/Zaib-un-Nissa_Hami...
184642,http://dbpedia.org/resource/Zenna_Henderson
184905,http://dbpedia.org/resource/Zoe_Fairbairns
185086,http://dbpedia.org/resource/Ásta_Kristjana_Sve...


899 intellectuels en lien avec le féminisme 

### Sortir la liste

In [24]:
# Nettoyage de la liste de noms pour affichage

names_clean['person'] = df['person'].apply(lambda x : x.replace('http://dbpedia.org/resource/', '').replace('_',' '))

names_clean

,person
212,A. P. de Zoysa
480,Aasta Hansteen
835,Abdullah Öcalan
1540,Adam Lewis Bingaman
1701,Addie Wagenknecht
...,...
184439,Zaib-un-Nissa Hamidullah
184642,Zenna Henderson
184905,Zoe Fairbairns
185086,Ásta Kristjana Sveinsdóttir


In [25]:
fem_list = names_clean.values.tolist()
print(fem_list)

[['A. P. de Zoysa'], ['Aasta Hansteen'], ['Abdullah Öcalan'], ['Adam Lewis Bingaman'], ['Addie Wagenknecht'], ['Adele Reinhartz'], ['Adnan Adıvar'], ['Adolphe Monod'], ['Adriana Cavarero'], ['Adrianne Wadewitz'], ['Adrienne Rich'], ['Agnes Maule Machar'], ['Agnès Varda'], ['Ahmad bey Javanshir'], ['Ailsa McKay'], ['Akasha Gloria Hull'], ['Aletta Norval'], ['Alexandra Juhasz'], ['Alexis Hunter'], ['Alia Al-Saji'], ['Alice Baber'], ['Alice Bach'], ['Alice Beck Kehoe'], ['Alice Clark'], ['Alice Duer Miller'], ['Alice Gerstenberg'], ['Alice Manfield'], ['Alice Mary Robertson'], ['Alice Moore Hubbard'], ['Alice Paul'], ['Alice S. Rossi'], ['Alice Shields'], ['Alice Stone Blackwell'], ['Alice Walker'], ['Alice Wolfson'], ['Alicia Ostriker'], ['Aline Mackinnon'], ['Alison Donnell'], ['Alison Jaggar'], ['Alison Spedding'], ['Alison Wylie'], ['Alix Dobkin'], ['Alix Kates Shulman'], ['Alix Olson'], ['Allette Brooks'], ['Allyson Mitchell'], ['Amalia Holst'], ['Amalia Mesa-Bains'], ['Amanda Marcot

Pour toutes les personnes de la liste ci-dessus, les termes feminist ou feminism apparaissent dans leur résumé wikipédia (en anglais).

### Avec les dates de naissances

In [26]:
abstract = """
WITH tw0 AS(
SELECT DISTINCT uri_entity ,
CASE WHEN value like 'http%'
then replace(value, 'http://dbpedia.org/resource/', '')
else value END abstract
FROM
	property
WHERE
	property IN ('http://dbpedia.org/ontology/abstract')
), tw1 AS (
SELECT GROUP_CONCAT(value) birth_dates, uri_entity, count(*) as eff
FROM property p
WHERE p.property LIKE '%birthD%' OR p.property LIKE '%birthY%'
GROUP BY uri_entity)
SELECT tw1.uri_entity, tw1.birth_dates, abstract
FROM tw1, tw0
WHERE tw1.uri_entity = tw0.uri_entity;
"""

In [27]:
# Connnexion à la base de données
cn = sql.connect(db)
c = cn.cursor()

### Exécuter la requête sur la base de donées SQLite pour récupérer les valeurs que contient la ligne
c.execute(abstract)
result_q = []
result_q = c.fetchall()

# Fermer la connexion et afficher les cinq premiers résultats
cn.close()
print(len(result_q))
result_q[:5]

103825


[("http://dbpedia.org/resource/'Abd_al-'Aziz_al-Wafa'i",
  '0811,811,811',
  "Abd al-'Aziz ibn Muhammad al-Qahiri al-Wafa'i known as Ibn al-Aqba'i (1408–1471) was an Egyptian astronomer and mathematician in the 15th century. He was born in 811 H.E. (1408 AD) and died in 876 H.E (1471 AD). Some sources say that he died in 874 or 879 H.E. He was a student of Ibn al-Majdi and . He became a time keeper at the Mu'ayyad Mosque in Cairo. He also became the chief of the time keepers of the Azhar mosque and the Maridani Mosque. He was interested in the calculation of the local times of cities based on their latitude and longitude. He invented an astronomical device called Da'irah al-Mu'addal. He wrote about 40 books and 20 treatises on mathematics and astronomy, none of them published. Among his works is the manuscript Risalah al-'Amal bi al-Rub' al-Mujayyab"),
 ('http://dbpedia.org/resource/14th_Dalai_Lama',
  '1935,1935-07-06',
  'The 14th Dalai Lama (religious name: Tenzin Gyatso, shortened 

In [29]:
df_abs_birth = pd.DataFrame(result_q, columns=['person','birth','abstract'])
df_abs_birth

,person,birth,abstract
0,http://dbpedia.org/resource/'Abd_al-'Aziz_al-W...,"0811,811,811",Abd al-'Aziz ibn Muhammad al-Qahiri al-Wafa'i ...
1,http://dbpedia.org/resource/14th_Dalai_Lama,"1935,1935-07-06",The 14th Dalai Lama (religious name: Tenzin Gy...
2,http://dbpedia.org/resource/5th_Dalai_Lama,"1617,1617",Ngawang Lobsang Gyatso (Tibetan: ངག་དབང་བློ་བཟ...
3,http://dbpedia.org/resource/9th_Wonder,"1975-01-15,1975,1975-01-15","Patrick Denard Douthit (born January 15, 1975)..."
4,http://dbpedia.org/resource/A'Lelia_Bundles,"1952-06-07,1952,1952,1952-06-07","A'Lelia Bundles (born June 7, 1952) is an Amer..."
...,...,...,...
103820,http://dbpedia.org/resource/Ștefan_Gușă,"1940-04-17,1940,1940-04-17",Ștefan Gușă or Gușe (17 April 1940 – 28 March ...
103821,http://dbpedia.org/resource/Ștefan_Luchian,"1868-02-01,1868,1868-02-01",Ștefan Luchian (Romanian pronunciation: [ʃteˈf...
103822,http://dbpedia.org/resource/Ștefan_Procopiu,"1890-01-19,1890,1890-01-19",Ștefan Procopiu (Romanian pronunciation: [ʃteˈ...
103823,http://dbpedia.org/resource/Ștefan_Secăreanu,"1959-01-08,1959,1959-01-08","Ștefan Secăreanu (born January 8, 1959, Chiose..."


In [36]:
searchfor = ['feminist', 'feminism']
contain_feminism = df_abs_birth[df_abs_birth['abstract'].str.contains('|'.join(searchfor))]
contain_feminism

,person,birth,abstract
462,http://dbpedia.org/resource/Abdullah_Öcalan,"1949-04-04,1949,1949,1949-04-04",Abdullah Öcalan ( OH-jə-lahn; Turkish: [œdʒaɫa...
924,http://dbpedia.org/resource/Addie_Wagenknecht,"1981-04-15,1981,1981-04-15","Addie Wagenknecht (born Portland, Oregon) is a..."
1001,http://dbpedia.org/resource/Adnan_Adıvar,"1882,1882,1882",Abdülhak Adnan Adıvar (1882 – 1 July 1955) was...
1173,http://dbpedia.org/resource/Adriana_Cavarero,"1947,1947","Adriana Cavarero (born 1947 in Bra, Italy) is ..."
1180,http://dbpedia.org/resource/Adrianne_Wadewitz,"1977-01-06,1977,1977,1977-01-06","Adrianne Wadewitz (January 6, 1977 – April 8, ..."
...,...,...,...
103175,http://dbpedia.org/resource/Zaib-un-Nissa_Hami...,"1918-12-25,1918,1918,1918-12-25",Zaib-un-Nissa (Bengali: জেবুন্নেসা হামিদুল্লাহ...
103293,http://dbpedia.org/resource/Zenna_Henderson,"1917-11-01,1917,1917-11-01","Zenna Chlarson Henderson (November 1, 1917 – M..."
103437,http://dbpedia.org/resource/Zoe_Fairbairns,"1948-12-20,1948,1948,1948-12-20",Zoe Fairbairns (born 1948) is a British femini...
103539,http://dbpedia.org/resource/Ásta_Kristjana_Sve...,"1969-10-05,1969,1969,1969-10-05","Ásta Kristjana Sveinsdóttir (born October 5, 1..."


REMARQUE : en ajoutant les années de naissance, on passe de 899 personnes à 530 (les autres n'ont pas leur année de naissance renseignée)

In [38]:
### nettoyage des noms

names_clean = pd.DataFrame(contain_feminism)
names_clean['person'] = names_clean['person'].apply(lambda x : x.replace('http://dbpedia.org/resource/', '').replace('_',' '))

names_clean

,person,birth,abstract
462,Abdullah Öcalan,"1949-04-04,1949,1949,1949-04-04",Abdullah Öcalan ( OH-jə-lahn; Turkish: [œdʒaɫa...
924,Addie Wagenknecht,"1981-04-15,1981,1981-04-15","Addie Wagenknecht (born Portland, Oregon) is a..."
1001,Adnan Adıvar,"1882,1882,1882",Abdülhak Adnan Adıvar (1882 – 1 July 1955) was...
1173,Adriana Cavarero,"1947,1947","Adriana Cavarero (born 1947 in Bra, Italy) is ..."
1180,Adrianne Wadewitz,"1977-01-06,1977,1977,1977-01-06","Adrianne Wadewitz (January 6, 1977 – April 8, ..."
...,...,...,...
103175,Zaib-un-Nissa Hamidullah,"1918-12-25,1918,1918,1918-12-25",Zaib-un-Nissa (Bengali: জেবুন্নেসা হামিদুল্লাহ...
103293,Zenna Henderson,"1917-11-01,1917,1917-11-01","Zenna Chlarson Henderson (November 1, 1917 – M..."
103437,Zoe Fairbairns,"1948-12-20,1948,1948,1948-12-20",Zoe Fairbairns (born 1948) is a British femini...
103539,Ásta Kristjana Sveinsdóttir,"1969-10-05,1969,1969,1969-10-05","Ásta Kristjana Sveinsdóttir (born October 5, 1..."


In [39]:
# Extraire les années
listeDateNaiss = []
annee = re.compile(r'\d{4}')

for l in result_q:
    listeDateNaiss.append(annee.findall(l[0])) 

In [41]:
# Extraire l'année de chaque ligne.
names_clean['anneeNaiss'] = names_clean['birth'].apply(lambda row: annee.findall(row)[0] if annee.findall(row) else None)  
# Affiche les cinq premiers résultats
names_clean[:5]

,person,birth,abstract,anneeNaiss
462,Abdullah Öcalan,"1949-04-04,1949,1949,1949-04-04",Abdullah Öcalan ( OH-jə-lahn; Turkish: [œdʒaɫa...,1949
924,Addie Wagenknecht,"1981-04-15,1981,1981-04-15","Addie Wagenknecht (born Portland, Oregon) is a...",1981
1001,Adnan Adıvar,"1882,1882,1882",Abdülhak Adnan Adıvar (1882 – 1 July 1955) was...,1882
1173,Adriana Cavarero,"1947,1947","Adriana Cavarero (born 1947 in Bra, Italy) is ...",1947
1180,Adrianne Wadewitz,"1977-01-06,1977,1977,1977-01-06","Adrianne Wadewitz (January 6, 1977 – April 8, ...",1977


In [43]:
# Crée un tableau montrant 
df = names_clean[['person', 'abstract', 'anneeNaiss']]
print(len(df))
df

530


,person,abstract,anneeNaiss
462,Abdullah Öcalan,Abdullah Öcalan ( OH-jə-lahn; Turkish: [œdʒaɫa...,1949
924,Addie Wagenknecht,"Addie Wagenknecht (born Portland, Oregon) is a...",1981
1001,Adnan Adıvar,Abdülhak Adnan Adıvar (1882 – 1 July 1955) was...,1882
1173,Adriana Cavarero,"Adriana Cavarero (born 1947 in Bra, Italy) is ...",1947
1180,Adrianne Wadewitz,"Adrianne Wadewitz (January 6, 1977 – April 8, ...",1977
...,...,...,...
103175,Zaib-un-Nissa Hamidullah,Zaib-un-Nissa (Bengali: জেবুন্নেসা হামিদুল্লাহ...,1918
103293,Zenna Henderson,"Zenna Chlarson Henderson (November 1, 1917 – M...",1917
103437,Zoe Fairbairns,Zoe Fairbairns (born 1948) is a British femini...,1948
103539,Ásta Kristjana Sveinsdóttir,"Ásta Kristjana Sveinsdóttir (born October 5, 1...",1969


In [45]:
# Crée un tableau montrant les entités, leurs écoles philosophiques et leur année de naissance
df_person_naissance = names_clean[['person', 'anneeNaiss']]
print(len(df_person_naissance))
df_person_naissance

530


,person,anneeNaiss
462,Abdullah Öcalan,1949
924,Addie Wagenknecht,1981
1001,Adnan Adıvar,1882
1173,Adriana Cavarero,1947
1180,Adrianne Wadewitz,1977
...,...,...
103175,Zaib-un-Nissa Hamidullah,1918
103293,Zenna Henderson,1917
103437,Zoe Fairbairns,1948
103539,Ásta Kristjana Sveinsdóttir,1969


In [49]:
# anneeNaiss dans l'ordre croissant

df_naissance_tri = df_person_naissance.sort_values(by='anneeNaiss', ascending=True)
df_naissance_tri

,person,anneeNaiss
35202,Gita Sahgal,0007
41063,Hypatia,0350
59444,Laura Cereta,1469
64476,Marie Meurdrac,1610
54887,Juana Inés de la Cruz,1648
...,...,...
59651,Laurie Penny,1986
22314,Deepika Padukone,1986
60234,Lena Chen,1987
71227,Nadezhda Tolokonnikova,1989


530 intellectuels en lien avec le féminisme avec l'année de naissance renseignée

### Sortir la liste

In [51]:
fem_list = df_naissance_tri.values.tolist()
print(fem_list)

[['Gita Sahgal', '0007'], ['Hypatia', '0350'], ['Laura Cereta', '1469'], ['Marie Meurdrac', '1610'], ['Juana Inés de la Cruz', '1648'], ['Damaris Cudworth Masham', '1659'], ['Françoise de Graffigny', '1695'], ['Judith Sargent Murray', '1751'], ['William Godwin', '1756'], ['Mary Wollstonecraft', '1759'], ['Charles Fourier', '1772'], ['Yu Zhengxie', '1775'], ['Frances Wright', '1795'], ['John Stuart Mill', '1806'], ['Johann Jakob Bachofen', '1815'], ['George Henry Lewes', '1817'], ['Lucy Stone', '1818'], ['Joseph Wolstenholme', '1829'], ['Clémence Royer', '1830'], ['Lucy Stanton (abolitionist)', '1831'], ['Soledad Acosta', '1833'], ['Anna Brackett', '1836'], ['Agnes Maule Machar', '1837'], ['Sophia Jex-Blake', '1840'], ['Louis Massebieau', '1840'], ['Ellen Swallow Richards', '1842'], ['Sir Charles Dilke, 2nd Baronet', '1843'], ['Millicent Fawcett', '1847'], ['Princess Louise, Duchess of Argyll', '1848'], ['Sophie Bryant', '1850'], ['Knut Wicksell', '1851'], ['Léopold Lacour', '1854'], ['

Grâce à cette liste, nous pouvons par exemple voir quelles sont les personnes considérés comme intellectuelles qui ont un lien avec le féminisme et voir parmi celles-ci quelles sont les plus anciennes. On peut ensuite taper leur nom sur Wikipédia pour en savoir plus.